In [1]:

import cv2

from src.diagram.description_models import *
from src.diagram.ocr.model import *
from src.diagram.struct.model import *


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from rich import print as pp

In [4]:


img = cv2.imread(
    "/home/petr/projects/mltests/dataset/out_image/0e5e7c91e225dd48e344d35c90e62fa7e867890268dd94df219d8b2eff0356aa.0.jpg")
detector_out = DetectorOutput.model_validate_json(open("a.json").read())
ocr_out = OCROutput.model_validate_json(open("b.json").read())

In [5]:
from src.diagram.annotate.diagram import DiagramElementsGenerator

contents = DiagramElementsGenerator(detector_out)()

In [6]:
def iou_metrics(inner, outer):
    xA = max(inner[0], outer[0])
    yA = max(inner[1], outer[1])
    xB = min(inner[2], outer[2])
    yB = min(inner[3], outer[3])
    intersect = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    a_area = (inner[2] - inner[0] + 1) * (inner[3] - inner[1] + 1)
    b_area = (outer[2] - outer[0] + 1) * (outer[3] - outer[1] + 1)
    return dict(
        intersect=intersect,
        a_area=a_area,
        b_area=b_area,
        iou=intersect / float(a_area + b_area - intersect),  # сколько занимает пересечение от объединения
        inters_over_inner=intersect / a_area
        # сколько занимает площадь пересечения относительно площади внутреннего (inner) - 1.0 показывает что весь inner в outer
    )


def rank(data, key, desc=True):
    data = [(key(i), i) for i in data]
    return sorted(data, key=lambda i: i[0], reverse=desc)

In [7]:


class Labeler:
    def __init__(self, detector_data: DetectorOutput, ocr_data: OCROutput):
        self.detector_data = detector_data
        self.ocr_data = ocr_data
        self.label_pool = {i.bbox: i.text for i in self.ocr_data.texts}

    def resolve_internal_labels(self, bbox):
        if not len(self.label_pool): return None
        variants = rank(self.label_pool.items(), lambda x: iou_metrics(x[0], bbox)['inters_over_inner'], desc=True)
        variants = [i for r, i in variants if r > 0.9]
        # собираем текст в порядке чтения l->r, up->down
        variants = rank(variants, key=lambda i: i[0][1] + i[0][0] * 5000, desc=False)
        res = ""
        for r, (bb, txt) in variants:
            res += txt + " "
            self.label_pool.pop(bb)  # убираем из дальнейшего рассмотрения
        return res.strip() or None

    def resolve_external_label(self, bbox):
        return None

    def resolve_external_label_for_line(self, line):
        return None

    def resolve_label_for_process(self, obj):
        return None

    def resolve_label_for_pool(self, obj):
        return None

    def run(self, diag: DiagramContents) -> DiagramContents:
        out = diag.model_copy()
        for i in out.elements:
            if i.type == GBPMNElementType.TASK:
                i.label = self.resolve_internal_labels(i.bbox)
            if i.type == GBPMNElementType.VIRT_LANE:
                i.label = self.resolve_label_for_pool(i.bbox)
            if i.type == GBPMNElementType.VIRT_PROC:
                i.label = self.resolve_label_for_process(i.bbox)
            if i.type in {GBPMNElementType.GATEWAY, GBPMNElementType.EVENT_START, GBPMNElementType.EVENT_END,
                          GBPMNElementType.EVENT_CATCH, GBPMNElementType.EVENT_THROW}:
                i.label = self.resolve_external_label(i.bbox)
        for i in out.links:
            i.label = self.resolve_external_label_for_line(i.line)
        return out


x = Labeler(detector_out, ocr_out)
contents = x.run(contents)

In [8]:
from uuid import uuid4
from src.diagram.description_models import DiagramContents

AGGREGATED_ETYPES = {GBPMNElementType.VIRT_LANE, GBPMNElementType.VIRT_PROC}


class DiagramNestBinder:

    def __init__(self, data: DiagramContents):
        self.data = data.model_copy()
        self.basic_elements = [i
                               for i in self.data.elements
                               if i.type not in AGGREGATED_ETYPES]

    def __scan_internals(self, bbox, source=None, cutoff=0.9):
        variants = rank(source or self.basic_elements,
                        lambda x: iou_metrics(x.bbox, bbox)['inters_over_inner'],
                        desc=True)
        return [i for r, i in variants if r > cutoff]

    def __processes(self):
        return [i for i in self.data.elements if i.type == GBPMNElementType.VIRT_PROC]

    def __lanes(self):
        return [i for i in self.data.elements if i.type == GBPMNElementType.VIRT_LANE]

    def __call__(self) -> DiagramContents:
        # находим лэйны в процессах
        # добавляем единственный дефолтный лэйн если ни одного нет
        # для лэйнов находим содержимое
        # связываем процессы, лэйны и содержимое
        for proc in self.__processes():
            lanes = self.__scan_internals(proc.bbox, source=self.__lanes(), cutoff=0.7)
            if not lanes:
                fallback_lane = GBPMNElement(
                    id=str(uuid4()),
                    label=proc.label,
                    type=GBPMNElementType.VIRT_LANE,
                    bbox=proc.bbox,
                )
                lanes = [fallback_lane]

            lids = []
            for lane in lanes:
                lane_content = self.__scan_internals(lane.bbox, source=self.basic_elements, cutoff=0.9)
                lane = GBPMNLaneElement(**lane.model_dump(),
                                        process_id=proc.id,
                                        nested_ids=[i.id for i in lane_content])
                lids.append(lane.id)
                self.data.drop(lane.id)
                self.data.add(lane)

            proc = GBPMNProcessElement(**proc.model_dump(), lanes_ids=lids)
            self.data.drop(proc.id)
            self.data.add(proc)

        return self.data


contents = DiagramNestBinder(contents)()
pp(contents.elements)

[
    GBPMNElement(
        id='68f858b5-6e4a-4cdf-a247-170c50e53a31',
        label='Display report',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(884, 215, 984, 294)
    ),
    GBPMNElement(
        id='102cea39-31f2-4042-8fbb-e01ed8857fd1',
        label='Process request',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(150, 589, 250, 669)
    ),
    GBPMNElement(
        id='051b6e9f-59dd-40ae-af80-19bd3b47616e',
        label='Return result to clerk',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(580, 334, 679, 415)
    ),
    GBPMNElement(
        id='aaca54b0-f1db-43b1-a7f9-4b7b3fcf78ff',
        label='Inform clerk of delay',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(480, 214, 580, 295)
    ),
    GBPMNElement(
        id='826591fb-5e81-4b91-a359-1699dd46be4d',
        label='Retum level report',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(470, 589, 571, 669)
    ),
    GBPMNElement(
        id='ca9ddacf-eace-4f64-82be-78a556cfee44',
        label='Process level scoring',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(510, 704, 610, 785)
    ),
    GBPMNElement(
        id='58a31305-54e8-419c-8ac7-3f52c8d69bc7',
        label='Retum report evei',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(655, 705, 754, 785)
    ),
    GBPMNElement(
        id='5da82d1e-0c05-4f34-9a70-900b73d80369',
        label='Delay; send message',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(264, 704, 365, 785)
    ),
    GBPMNElement(
        id='ccb85d01-4125-4455-a4ad-39fc682d7fff',
        label='Send level request',
        type=TASK,
        subtype=TASK_OTHER,
        bbox=(135, 274, 235, 354)
    ),
    GBPMNElement(
        id='682e2d56-38a3-47d1-b0f2-4ec417a1e5fa',
        label=None,
        type=GATEWAY,
        subtype=GATEWAY_EXCLUSIVE,
        bbox=(294, 610, 334, 650)
    ),
    GBPMNElement(
        id='dfa7a2d7-c8c0-468c-b584-d469a60192a1',
        label=None,
        type=GATEWAY,
        subtype=GATEWAY_EXCLUSIVE,
        bbox=(279, 294, 319, 335)
    ),
    GBPMNElement(
        id='75e5d2e3-e94f-4648-97fe-a8d71abfab72',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(60, 300, 89, 329)
    ),
    GBPMNElement(
        id='26b32036-ef03-49eb-8cea-e7b24479e936',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(60, 614, 89, 644)
    ),
    GBPMNElement(
        id='a8c69f18-e12d-4ef8-8dd2-7c175e87d229',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(415, 729, 444, 760)
    ),
    GBPMNElement(
        id='e8bc4ec8-bf26-461d-8ee6-6ce44cadf3f2',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(735, 360, 762, 388)
    ),
    GBPMNElement(
        id='01552c3d-51ac-4150-8413-ea7e36b0c8ff',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(330, 359, 359, 390)
    ),
    GBPMNElement(
        id='df56b1ed-d225-4a8a-9673-bd0f473c0b6a',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(810, 729, 839, 760)
    ),
    GBPMNElement(
        id='1725a400-3380-4de2-bd5a-cc5090e97898',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(415, 240, 444, 270)
    ),
    GBPMNElement(
        id='08153965-f3c3-4b07-b0e0-8d2e32fdca09',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(810, 240, 839, 270)
    ),
    GBPMNElement(
        id='a3b20dce-e8ee-49c0-b74e-80270e1681b5',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(689, 616, 717, 643)
    ),
    GBPMNElement(
        id='23fb21ff-9574-4c48-9c05-8ae08f2f8fbe',
        label=None,
        type=EVENT_START,
        subtype=EVENT_OTHER,
        bbox=(1030, 240, 1057, 268)
    ),
    GBPMNElement(
        id='5930e1

In [9]:
from src.diagram.annotate.tools import dist_pt2bbox

MAX_LINK2OBJ_DISTANCE = 10


class DiagramLinkBinder:

    def __init__(self, data: DiagramContents):
        self.data = data.model_copy()
        self.linkable = [i
                         for i in self.data.elements
                         if i.type not in {GBPMNElementType.VIRT_LANE}]

    def __search_near(self, pos):
        variants = rank(self.linkable, lambda x: dist_pt2bbox(pos, x.bbox), desc=False)
        variants = [i for r, i in variants if r < MAX_LINK2OBJ_DISTANCE]
        return variants[0] if variants else None

    def __call__(self) -> DiagramContents:
        old_count = len(self.data.links)
        links = []
        for i in self.data.links:
            if i.source_id and i.target_id: continue
            s, t = self.__search_near(i.line[0]), self.__search_near(i.line[-1])
            if not s or not t: continue
            i.source_id = s.id
            i.target_id = t.id
            links.append(i)
        self.data.links = links
        new_count = len(links)
        if new_count != old_count:
            print(f"links dropped {old_count} -> {new_count}")
        return self.data


contents = DiagramLinkBinder(contents)()
pp(contents.links)

links dropped 32 -> 27


[
    GBPMNFlow(
        id='65608ef4-8e67-4739-b33a-d589e3efc4f1',
        label=None,
        type=MESSAGE,
        source_id='ccb85d01-4125-4455-a4ad-39fc682d7fff',
        target_id='26b32036-ef03-49eb-8cea-e7b24479e936',
        line=[(174, 352), (175, 441), (77, 441), (75, 615)]
    ),
    GBPMNFlow(
        id='c5f522ab-c1e3-4ced-b284-9b2589688956',
        label=None,
        type=SEQUENCE,
        source_id='dfa7a2d7-c8c0-468c-b584-d469a60192a1',
        target_id='1725a400-3380-4de2-bd5a-cc5090e97898',
        line=[(299, 294), (300, 255), (413, 254)]
    ),
    GBPMNFlow(
        id='72479269-8139-4121-b8d4-6e389a11208c',
        label=None,
        type=SEQUENCE,
        source_id='dfa7a2d7-c8c0-468c-b584-d469a60192a1',
        target_id='01552c3d-51ac-4150-8413-ea7e36b0c8ff',
        line=[(300, 335), (299, 374), (329, 375)]
    ),
    GBPMNFlow(
        id='46309c02-cd16-4b2c-928f-dc4a73a209ee',
        label=None,
        type=SEQUENCE,
        source_id='102cea39-31f2-4042-8fbb-e01ed8857fd1',
        target_id='682e2d56-38a3-47d1-b0f2-4ec417a1e5fa',
        line=[(250, 630), (294, 630)]
    ),
    GBPMNFlow(
        id='ba6e22fe-c4f1-41d9-99ea-da9598942612',
        label=None,
        type=SEQUENCE,
        source_id='826591fb-5e81-4b91-a359-1699dd46be4d',
        target_id='7d93d555-7cb5-4005-95e5-a1288be63e00',
        line=[(571, 630), (615, 630)]
    ),
    GBPMNFlow(
        id='7f3e8c3c-a344-423f-968a-edca10bf6630',
        label=None,
        type=SEQUENCE,
        source_id='7d93d555-7cb5-4005-95e5-a1288be63e00',
        target_id='a3b20dce-e8ee-49c0-b74e-80270e1681b5',
        line=[(644, 630), (689, 629)]
    ),
    GBPMNFlow(
        id='018a894a-b6cd-4662-a7c5-1e3551c1a3de',
        label=None,
        type=SEQUENCE,
        source_id='051b6e9f-59dd-40ae-af80-19bd3b47616e',
        target_id='e8bc4ec8-bf26-461d-8ee6-6ce44cadf3f2',
        line=[(679, 374), (734, 374)]
    ),
    GBPMNFlow(
        id='7aa42bdc-f192-4587-b9e6-b4983a7e1905',
        label=None,
        type=SEQUENCE,
        source_id='26b32036-ef03-49eb-8cea-e7b24479e936',
        target_id='102cea39-31f2-4042-8fbb-e01ed8857fd1',
        line=[(89, 630), (150, 630)]
    ),
    GBPMNFlow(
        id='6b041e91-e505-4ab1-801e-5e2e45b8f238',
        label=None,
        type=SEQUENCE,
        source_id='75e5d2e3-e94f-4648-97fe-a8d71abfab72',
        target_id='ccb85d01-4125-4455-a4ad-39fc682d7fff',
        line=[(90, 315), (135, 314)]
    ),
    GBPMNFlow(
        id='0cd49370-7a33-4b06-9a30-293269401e27',
        label=None,
        type=SEQUENCE,
        source_id='ccb85d01-4125-4455-a4ad-39fc682d7fff',
        target_id='dfa7a2d7-c8c0-468c-b584-d469a60192a1',
        line=[(235, 315), (280, 315)]
    ),
    GBPMNFlow(
        id='67986e69-9c50-42b9-a8a1-c96bb44957e9',
        label=None,
        type=SEQUENCE,
        source_id='1725a400-3380-4de2-bd5a-cc5090e97898',
        target_id='aaca54b0-f1db-43b1-a7f9-4b7b3fcf78ff',
        line=[(444, 254), (479, 254)]
    ),
    GBPMNFlow(
        id='95288e5c-2156-4a04-ad31-2fef7400103e',
        label=None,
        type=SEQUENCE,
        source_id='68f858b5-6e4a-4cdf-a247-170c50e53a31',
        target_id='23fb21ff-9574-4c48-9c05-8ae08f2f8fbe',
        line=[(985, 255), (1030, 254)]
    ),
    GBPMNFlow(
        id='75dec569-c449-4000-a8f5-0f77df4ef1f9',
        label=None,
        type=MESSAGE,
        source_id='7d93d555-7cb5-4005-95e5-a1288be63e00',
        target_id='01552c3d-51ac-4150-8413-ea7e36b0c8ff',
        line=[(630, 612), (626, 438), (352, 436), (346, 399)]
    ),
    GBPMNFlow(
        id='c7056134-e260-4bd2-b7da-90cf36c402df',
        label=None,
        type=SEQUENCE,
        source_id='ca9ddacf-eace-4f64-82be-78a556cfee44',
        target_id='58a31305-54e8-419c-8ac7-3f52c8d69bc7',
        line=[(609, 744), (632, 745), (655, 744)]
    ),
    GBPMNFlow(
        id='1fcfe9e9-5677-40ab-8c25-d877056150fd',
        label=None,
        type=SEQUENCE,
        source_id='5da82d1e-0c

In [10]:
from src.diagram.description_models import DiagramContents, GBPMNDiagram


class DiagramBuilder:
    def __init__(self, data: DiagramContents):
        self.data = data

    def __elements(self, type):
        return [i for i in self.data.elements if i.type == type]

    def __call__(self) -> GBPMNDiagram:
        processes = []
        for p in self.__elements(GBPMNElementType.VIRT_PROC):
            lanes = [
                GBPMNLane(
                    id=l.id,
                    label=l.label,
                    bbox=l.bbox,
                    objects=[
                        o
                        for o in self.data.elements
                        if o.type not in {GBPMNElementType.VIRT_LANE,
                                          GBPMNElementType.VIRT_PROC} \
                           and o.id in l.nested_ids
                    ]
                )
                for l in self.__elements(GBPMNElementType.VIRT_LANE)
                if l.process_id == p.id
            ]
            processes.append(GBPMNProcess(
                id=p.id, label=p.label, bbox=p.bbox, lanes=lanes
            ))

        return GBPMNDiagram(
            processes=processes,
            flows={(i.source_id, i.target_id): i for i in self.data.links},
            objects={i.id: i for i in self.data.elements},
        )


bpmnd = DiagramBuilder(contents)()
pp(bpmnd)

GBPMNDiagram(
    processes=[
        GBPMNProcess(
            id='ae743150-2b51-4ac7-9fdf-4f39a238c978',
            label=None,
            bbox=(4, 570, 1148, 820),
            lanes=[
                GBPMNLane(
                    id='a646ca5c-91bc-4f7b-bb16-62c257904f00',
                    label=None,
                    bbox=(26, 570, 1148, 820),
                    objects=[
                        GBPMNElement(
                            id='102cea39-31f2-4042-8fbb-e01ed8857fd1',
                            label='Process request',
                            type=TASK,
                            subtype=TASK_OTHER,
                            bbox=(150, 589, 250, 669)
                        ),
                        GBPMNElement(
                            id='826591fb-5e81-4b91-a359-1699dd46be4d',
                            label='Retum level report',
                            type=TASK,
                            subtype=TASK_OTHER,
                            bbox=(470, 589, 571, 669)
                        ),
                        GBPMNElement(
                            id='ca9ddacf-eace-4f64-82be-78a556cfee44',
                            label='Process level scoring',
                            type=TASK,
                            subtype=TASK_OTHER,
                            bbox=(510, 704, 610, 785)
                        ),
                        GBPMNElement(
                            id='58a31305-54e8-419c-8ac7-3f52c8d69bc7',
                            label='Retum report evei',
                            type=TASK,
                            subtype=TASK_OTHER,
                            bbox=(655, 705, 754, 785)
                        ),
                        GBPMNElement(
                            id='5da82d1e-0c05-4f34-9a70-900b73d80369',
                            label='Delay; send message',
                            type=TASK,
                            subtype=TASK_OTHER,
                            bbox=(264, 704, 365, 785)
                        ),
                        GBPMNElement(
                            id='682e2d56-38a3-47d1-b0f2-4ec417a1e5fa',
                            label=None,
                            type=GATEWAY,
                            subtype=GATEWAY_EXCLUSIVE,
                            bbox=(294, 610, 334, 650)
                        ),
                        GBPMNElement(
                            id='26b32036-ef03-49eb-8cea-e7b24479e936',
                            label=None,
                            type=EVENT_START,
                            subtype=EVENT_OTHER,
                            bbox=(60, 614, 89, 644)
                        ),
                        GBPMNElement(
                            id='a8c69f18-e12d-4ef8-8dd2-7c175e87d229',
                            label=None,
                            type=EVENT_START,
                            subtype=EVENT_OTHER,
                            bbox=(415, 729, 444, 760)
                        ),
                        GBPMNElement(
                            id='df56b1ed-d225-4a8a-9673-bd0f473c0b6a',
                            label=None,
                            type=EVENT_START,
                            subtype=EVENT_OTHER,
                            bbox=(810, 729, 839, 760)
                        ),
                        GBPMNElement(
                            id='a3b20dce-e8ee-49c0-b74e-80270e1681b5',
                            label=None,
                            type=EVENT_START,
                            subtype=EVENT_OTHER,
                            bbox=(689, 616, 717, 643)
                        ),
                        GBPMNElement(
                            id='5930e1b2-5305-42a6-ab1d-982163961350',
                            label=None,
                            type=EVENT_START,
                            subtype=EVENT_OTHER,
                            bbox=(884, 730, 912, 759)
                        

In [11]:
from io import BytesIO
from matplotlib import pyplot as plt
import networkx as nx
import iplotx as ipx
from src.diagram.annotate.tools import bbox_center
from src.diagram.description_models import DiagramContents, GBPMNDiagram

G_NODE_COLOR = {
    GBPMNElementType.TASK: "yellow",
    GBPMNElementType.GATEWAY: "red",
    GBPMNElementType.EVENT_START: "green",
    GBPMNElementType.EVENT_END: "gray",
    GBPMNElementType.EVENT_CATCH: "blue",
    GBPMNElementType.EVENT_THROW: "cyan",
}


class GraphBuilder:
    def __init__(self, contents: DiagramContents, diagram: GBPMNDiagram):
        self.contents = contents
        self.diagram = diagram
        self.graph = nx.DiGraph()
        self.push_nodes()
        self.push_edges()

    def push_nodes(self):
        for p in self.diagram.processes:
            for l in p.lanes:
                for o in l.objects:
                    self.graph.add_node(o.id, **o.model_dump())

    def push_edges(self):
        for (s1, s2), e in self.diagram.flows.items():
            if e.type == GBPMNFlowType.SEQUENCE:
                self.graph.add_edge(s1, s2, **e.model_dump())

    def create_layout(self):
        pos = {
            i: bbox_center(v['bbox'])
            for i, v in self.graph.nodes.data()
        }
        return nx.spring_layout(self.graph, pos=pos, fixed=self.graph.nodes)

    def visualize(self, sz=(640, 480), dpi=80):
        vertex_color = [G_NODE_COLOR[v['type']] for _, v in self.graph.nodes.data()]
        layout = self.create_layout()
        fig, ax = plt.subplots(figsize=(sz[0] / dpi, sz[1] / dpi), dpi=dpi)
        ipx.plot(self.graph, ax=ax, layout=layout, vertex_facecolor=vertex_color)
        buf = BytesIO()
        fig.savefig(buf, format='png', dpi=dpi, bbox_inches='tight')
        buf.seek(0)
        png_bytes = buf.getvalue()
        plt.close(fig)
        return png_bytes

    def __call__(self) -> nx.DiGraph:
        return self.graph


# layout = nx.bfs_layout(graph, next(iter(graph.nodes())))
gb = GraphBuilder(contents, bpmnd)
graph = gb()

with open("test.png", "wb") as f:
    f.write(gb.visualize())


In [40]:
import numpy as np
import importlib


In [20]:
from src.renderer.renderbpmn import BPMNRenderer, save_all

renderer = BPMNRenderer()

In [46]:
all_pts = np.array(list(gb.create_layout().values()))
p_min, p_max = all_pts.min(axis=0), all_pts.max(axis=0)
b_range = p_max - p_min


(array([ 74.5, 254. ]), array([1043.5,  745. ]))

In [48]:
import src.renderer.codegen as codegen
importlib.reload(codegen)


cg = codegen.GraphBPMNCodegen()
code = cg(graph, gb.create_layout(), scale=2)
save_all(await renderer.render_by_code(code))
